# Segmenting and Clustering Neighborhoods in Toronto

## Week 1(Part 1)

**To start we first have to scrape data from Wikipedia as said in the instructions.**
We'll start this by :
    - Importing beautiful which is a `library` for scraping data over web.

## In this part we'll clean and put the data into CSV form

In [1]:
#importing all the required libraries
#! pip install bs4
# !pip install requests

import pandas as pd
from bs4 import BeautifulSoup as bsoup
import requests

In [2]:
# scraping data from wikipedia for extracting data of Torronto Canada
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
sc_data = requests.get(url).text

extracted_data = bsoup(sc_data, 'lxml')
print(extracted_data)

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"YEb63tsFvnTJDWVde@NcBQAAABM","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":1011037969,"wgRevisionId":1011037969,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Communications in Ontario","Postal c

In [3]:
## putting everything together, to put it in a dataframe

column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

content = extracted_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text.strip("\n").replace(']',"")
            i = i + 1
        elif i == 1:
            borough = td.text.strip("\n").replace(']',"")
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip("\n").replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

In [4]:
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1
df_toronto = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_toronto

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [5]:
df_toronto = df_toronto.dropna()
Null_values = 'Not assigned'
df_toronto = df_toronto[(df_toronto.Postalcode != Null_values ) & (df_toronto.Borough != Null_values) & (df_toronto.Neighborhood != Null_values)]

In [6]:
df_toronto.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df_toronto.groupby(['Postalcode', 'Borough'])
df = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [8]:
df.to_csv('toronto.csv', index=False)